In [ ]:
import pandas as pd
from pathlib import Path
from sklearn import model_selection
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras import models, layers, metrics

# ensure same network weights at initialisation between notebook runs
import numpy as np
np.random.seed(22) 

In [ ]:
concrete_data_file = Path('concrete_data.csv')
if concrete_data_file.is_file():
    print('Reading concrete_data.csv...')
    concrete_data = pd.read_csv('concrete_data.csv')
    print('Done.')
else:
    print('Downloading concrete data CSV file...')
    # Original Source: http://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength
    root_path = 'https://www.stefanfiott.com/machine-learning/'
    folder = 'concrete-compressive-strength-regression-model-using-keras/'
    concrete_data = pd.read_csv(root_path + folder + 'concrete_data.csv')
    print('Done.')
    print('Saving to concrete_data.csv file...')
    concrete_data.to_csv('concrete_data.csv', index=False)
    print('Done.')
    
print(concrete_data.head())

In [ ]:
predictors = concrete_data.iloc[:,0:8].values
outcomes = concrete_data.iloc[:,8].values

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
predictors_scaled = min_max_scaler.fit_transform(predictors)
predictors_scaled[:5,]

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(predictors, outcomes, test_size=0.33, random_state=22)
print('X_train {0}, y_train {1}'.format(X_train.shape, y_train.shape))
print('X_test {0}, y_test {1}'.format(X_test.shape, y_test.shape))

In [ ]:
X_train (690, 8), y_train (690,)
X_test (340, 8), y_test (340,)

In [ ]:
network = models.Sequential()
network.add(layers.Dense(10, activation='relu', input_shape=(X_train.shape[1], )))
network.add(layers.Dense(5, activation='relu'))
network.add(layers.Dense(1))
network.compile(optimizer='adam',
                loss='mean_squared_error')

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(16,8), sharex=True, sharey=True)
losses = []
for i in range(2):
    for j in range(5):
        network.fit(X_train, y_train, epochs=50, batch_size=128, verbose=0);
        pred_loss = network.evaluate(X_test, y_test, verbose=0)
        losses.append(pred_loss)
        preds = network.predict(X_test)
        axes[i,j].scatter(preds, y_test, alpha=0.2)
        axes[i,j].set_title('{0} epochs'.format((5*i+j+1)*50))
        axes[i,j].set_ylabel('Actual')
        axes[i,j].set_xlabel('Predicted')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(losses)
ax.set_title('Concrete Compressive Strength Regression Model Loss')
epochs = [str(i*50) for i in range(1, len(losses)+1)]
ax.set_xticks(range(len(losses)))
ax.set_xticklabels(epochs)
ax.set_xlabel('Epochs')
ax.set_ylabel('Mean Squared Error')
ax.text(len(losses)-2, losses[len(losses)-1]+10, 'Min MSE: {0:.2f}'.format(losses[len(losses)-1]));

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
ols_y_pred = regr.predict(X_test)

fig, ax = plt.subplots(1, 1, figsize=(3,4))
ax.scatter(ols_y_pred, y_test, alpha=0.2)
ax.set_title('OLS Model'.format((5*i+j+1)*50))
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')

print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test, ols_y_pred)))

In [ ]:
from sklearn.linear_model import ElasticNetCV

regr = ElasticNetCV(cv=5)
regr.fit(X_train, y_train) 

elastic_y_pred = regr.predict(X_test)

fig, ax = plt.subplots(1, 1, figsize=(3,4))
ax.scatter(elastic_y_pred, y_test, alpha=0.2)
ax.set_title('Elastic Net Model'.format((5*i+j+1)*50))
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')

print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test, elastic_y_pred)))

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), 
                   cv=5,
                   param_grid={'C': [1e0, 1e1, 1e2, 1e3],
                               'gamma': np.logspace(-2, 2, 5),
                               'epsilon': np.arange(0.1, 0.5, 0.1)})
svr.fit(X_train, y_train)
svr_y_pred = svr.best_estimator_.predict(X_test)
fig, ax = plt.subplots(1, 1, figsize=(3,4))
ax.scatter(svr_y_pred, y_test, alpha=0.2)
ax.set_title('SVR Model'.format((5*i+j+1)*50))
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')

print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test, svr_y_pred)))